# 訓練データとして用いるデータの詳細

In [1]:
import numpy as np
import datetime
import os
import copy
import seaborn as sns
%matplotlib inline
# Generate WindowFrame Function
from mymodule import window
import joblib

import pandas as pd

# Parametor about Sensor
SensorNum=1 # Sensor number
#SensorAxis='AccX' # Axis

# Define Data Name
DATE= str( datetime.date.today() )+'/'

# Parametor about " Data Path"
StorePath = "/media/takeyama/Transfer/02_ActivityResearch/"

ParticipantsNum="Participants07/"
Version = "01_First/"
Time = "01_First/"
CommonName = 'Participants07-First-01-'
LabelName = CommonName+'.csv'

TITLE = ParticipantsNum+Version

DataPath = StorePath+ParticipantsNum+Version+"RawData/"+Time

if not os.path.exists(StorePath+TITLE+'LabelData/'+Time): 
    os.makedirs(StorePath+TITLE+'LabelData/'+Time)
LabelDataPath=StorePath+TITLE+'LabelData/'+Time

if not os.path.exists(StorePath+TITLE+'dictionary/'+Time): 
    os.makedirs(StorePath+TITLE+'dictionary/'+Time)
DictionaryDataPath=StorePath+TITLE+'dictionary/'+Time

if not os.path.exists(StorePath+TITLE+'feature/'+Time): 
    os.makedirs(StorePath+TITLE+'feature/'+Time)
FeaturePath=StorePath+TITLE+'feature/'+Time

if not os.path.exists(StorePath+TITLE+'window/'+Time): 
    os.makedirs(StorePath+TITLE+'window/'+Time)
WindowDataPath=StorePath+TITLE+'window/'+Time

if not os.path.exists(StorePath+TITLE+DATE+'studyOutput/'+Time): 
    os.makedirs(StorePath+TITLE+DATE+'studyOutput/'+Time)
StudyOutputPath=StorePath+TITLE+DATE+'studyOutput/'+Time

if not os.path.exists(StorePath+TITLE+DATE+'parametor/'+Time): 
    os.makedirs(StorePath+TITLE+DATE+'parametor/'+Time)
ParametorPath=StorePath+TITLE+DATE+'parametor/'+Time

if not os.path.exists(StorePath+TITLE+DATE+'result/'+Time): 
    os.makedirs(StorePath+TITLE+DATE+'result/'+Time)
ResultPath=StorePath+TITLE+DATE+'result/'+Time

if not os.path.exists(StorePath+TITLE+DATE+'graph/'+Time): 
    os.makedirs(StorePath+TITLE+DATE+'graph/'+Time)
GraphPath=StorePath+TITLE+DATE+'graph/'+Time

# Define Function for SVM

In [2]:
def GetData(PATH, SensorNum, Axis, Data, wNum, sNum):
    SensorName =['01_LeftHand','02_RightHand','03_LeftLeg','04_RightLeg','05_West','06_Chest']
    DictName=os.listdir(PATH)
    PathName=[DictName[i] for i in range(len(DictName)) if SensorName[SensorNum-1]+'-'+Axis in DictName[i]
             if Data in DictName[i] if str(wNum).zfill(4) in DictName[i] if str(sNum).zfill(4) in DictName[i]]
    print PathName
    #print joblib.load(FeaturePath+PathName[0]).shape
    return joblib.load(PATH+PathName[0])

In [3]:
def GetFeatureVector(PATH, SensorNum, Axis, wNum, sNum):
    df = pd.DataFrame({
        #'AccX_AutoCorr':np.load(FeaturePath+DictName[0])['data'],
        'Energy':GetData(PATH, SensorNum, Axis, 'Energy', wNum, sNum),
        'Entropy':GetData(PATH, SensorNum, Axis, 'Entropy', wNum, sNum),
        #'Frequency':GetData(PATH, SensorNum, Axis, 'Frequency'),
        'Max':GetData(PATH, SensorNum, Axis, 'Max', wNum, sNum),
        'Mean':GetData(PATH, SensorNum, Axis, 'Mean', wNum, sNum),
        'Median':GetData(PATH, SensorNum, Axis, 'Median', wNum, sNum),
        'Min':GetData(PATH, SensorNum, Axis, 'Min', wNum, sNum),
        'Range':GetData(PATH, SensorNum, Axis, 'Range', wNum, sNum),
        'RMS':GetData(PATH, SensorNum, Axis, 'RMS', wNum, sNum),
        'StdDev':GetData(PATH, SensorNum, Axis, 'StdDev', wNum, sNum),
        'Var':GetData(PATH, SensorNum, Axis, 'Var', wNum, sNum),
        'Axis':[Axis]*GetData(PATH, SensorNum, Axis, 'Mean', wNum, sNum).size},
    )
    return df

In [4]:
def MakeWindowFrameLabel(LabelData1d, wWidth, sWidth):
    w=window()
    w.SetData('DataAddingLabel', LabelData1d)    
    wind=w.Compile(wWidth, sWidth)
    windoW=wind.reshape((len(wind),np.prod(wind.shape[1:])))

    LabelArray=np.array([np.unique(windoW[l,:])[
            np.argmax(np.array([len( np.where(windoW[l]==np.unique(windoW[l])[i])[0])for i in range(np.unique(windoW[l]).size)]))]
             for l in range(windoW.shape[0])
    ])
    return LabelArray

In [5]:
def MakeTimeDomainLabelData(DataName,StartTime,GoalTime):
    # import LabelData -> pandas
    LabelDf = pd.read_csv(DataName,encoding='Shift-JIS')
    LabelData = LabelDf.as_matrix() #pandas -> ndarray
    
    # timedata hh.mm.ss -> ms
    TimeArray = pd.to_datetime( LabelDf['#Time'].values ) 
    t = np.array([ TimeArray[i].hour*3600*1000+TimeArray[i].minute*60*1000+TimeArray[i].second*1000+TimeArray[i].microsecond*0.001
                 for i in range( len(TimeArray) )]).astype(int)
    
    LabelData[:,0] = t.astype(int) #time data: unicode -> int
    # Time Series
    LabelArray = np.array([
        np.append( np.array(LabelData[i,0]+l) , LabelData[i,1:])
        for i in range(LabelData.shape[0]-1) 
        for l in range(LabelData[i+1,0]-LabelData[i,0])
    ])
    
    StartLabelArray = np.array([
        np.append( StartTime+i, LabelArray[0,1:])
        for i in range(LabelArray[0,0]-StartTime)
    ])
    
    GoalLabelArray = np.array([
        np.append( GoalTime+i, LabelArray[-1,1:])
        for i in range(1,1+GoalTime-LabelArray[-1,0])
    ])
    
    LabelArrayComp = np.vstack((StartLabelArray, LabelArray))
    LabelArrayComp = np.vstack((LabelArrayComp, GoalLabelArray))
    df = pd.DataFrame(LabelArrayComp)
    df.columns = LabelDf.columns
    
    return df.to_dict()

In [6]:
def WFLabelDataFrame(FileName,wNum,sNum):
    dictLabel=np.load(FileName)['data'][()]
    
    df = pd.DataFrame({
    '#Time':MakeWindowFrameLabel(np.array(dictLabel['#Time'].values()), wNum, sNum),
    'LeftHand-Verb':MakeWindowFrameLabel(np.array(dictLabel['LeftHand-Verb'].values()), wNum,sNum),
    'LeftHand-Object':MakeWindowFrameLabel(np.array(dictLabel['LeftHand-Object'].values()), wNum, sNum),
    'RightHand-Verb':MakeWindowFrameLabel(np.array(dictLabel['RightHand-Verb'].values()), wNum, sNum),
    'RightHand-Object ':MakeWindowFrameLabel(np.array(dictLabel['RightHand-Object'].values()), wNum, sNum),
    'Leg-Number':MakeWindowFrameLabel(np.array(dictLabel['Leg-Number'].values()), wNum, sNum),
    'Leg-Verb':MakeWindowFrameLabel(np.array(dictLabel['Leg-Object'].values()), wNum, sNum),
    'Leg-Object':MakeWindowFrameLabel(np.array(dictLabel['Leg-Object'].values()), wNum, sNum),
    'West':MakeWindowFrameLabel(np.array(dictLabel['West'].values()), wNum, sNum),
    'ActiveNumber':MakeWindowFrameLabel(np.array(dictLabel['ActiveNumber'].values()), wNum, sNum),
    })
    
    return df

このコピー元のスクリプトの結果からSVMによる識別はウィンドウ数が多い、ホウキや掃除機によってしまう  
ランダムフォレストを試してみる．そして，与えたデータの中で一番効果的な特徴量を調べてみる  

# Train Data

In [7]:
Train_AccX = GetFeatureVector(FeaturePath,1,'AccX', 256, 32)
Train_AccY = GetFeatureVector(FeaturePath,1,'AccY', 256, 32)
Train_AccZ = GetFeatureVector(FeaturePath,1,'AccZ', 256, 32)
TrainDataDF = pd.concat([Train_AccX,Train_AccY,Train_AccZ])
TrainDataDF = TrainDataDF.drop('Axis', axis=1)
TrainData = TrainDataDF.as_matrix()

['Participants07-First-01-01_LeftHand-AccX-win=0256-sld=0032_Energy.dump']
['Participants07-First-01-01_LeftHand-AccX-win=0256-sld=0032_Entropy.dump']
['Participants07-First-01-01_LeftHand-AccX-win=0256-sld=0032_Max.dump']
['Participants07-First-01-01_LeftHand-AccX-win=0256-sld=0032_Mean.dump']
['Participants07-First-01-01_LeftHand-AccX-win=0256-sld=0032_Median.dump']
['Participants07-First-01-01_LeftHand-AccX-win=0256-sld=0032_Min.dump']
['Participants07-First-01-01_LeftHand-AccX-win=0256-sld=0032_Range.dump']
['Participants07-First-01-01_LeftHand-AccX-win=0256-sld=0032_RMS.dump']
['Participants07-First-01-01_LeftHand-AccX-win=0256-sld=0032_StdDev.dump']
['Participants07-First-01-01_LeftHand-AccX-win=0256-sld=0032_Var.dump']
['Participants07-First-01-01_LeftHand-AccX-win=0256-sld=0032_Mean.dump']
['Participants07-First-01-01_LeftHand-AccY-win=0256-sld=0032_Energy.dump']
['Participants07-First-01-01_LeftHand-AccY-win=0256-sld=0032_Entropy.dump']
['Participants07-First-01-01_LeftHand-Ac

# Test Data

In [8]:
TestFeaturePath = '/media/takeyama/Transfer/02_ActivityResearch/Participants07/03_Third/feature/01_First/'
Test_AccX = GetFeatureVector(TestFeaturePath,1,'AccX', 256, 32)
Test_AccY = GetFeatureVector(TestFeaturePath,1,'AccY', 256, 32)
Test_AccZ = GetFeatureVector(TestFeaturePath,1,'AccZ', 256, 32)
TestDataDF = pd.concat([Test_AccX,Test_AccY,Test_AccZ])
TestDataDF = TestDataDF.drop('Axis', axis=1)
TestData = TestDataDF.as_matrix()

['Participants07-Third-01-01_LeftHand-AccX-win=0256-sld=0032_Energy.dump']
['Participants07-Third-01-01_LeftHand-AccX-win=0256-sld=0032_Entropy.dump']
['Participants07-Third-01-01_LeftHand-AccX-win=0256-sld=0032_Max.dump']
['Participants07-Third-01-01_LeftHand-AccX-win=0256-sld=0032_Mean.dump']
['Participants07-Third-01-01_LeftHand-AccX-win=0256-sld=0032_Median.dump']
['Participants07-Third-01-01_LeftHand-AccX-win=0256-sld=0032_Min.dump']
['Participants07-Third-01-01_LeftHand-AccX-win=0256-sld=0032_Range.dump']
['Participants07-Third-01-01_LeftHand-AccX-win=0256-sld=0032_RMS.dump']
['Participants07-Third-01-01_LeftHand-AccX-win=0256-sld=0032_StdDev.dump']
['Participants07-Third-01-01_LeftHand-AccX-win=0256-sld=0032_Var.dump']
['Participants07-Third-01-01_LeftHand-AccX-win=0256-sld=0032_Mean.dump']
['Participants07-Third-01-01_LeftHand-AccY-win=0256-sld=0032_Energy.dump']
['Participants07-Third-01-01_LeftHand-AccY-win=0256-sld=0032_Entropy.dump']
['Participants07-Third-01-01_LeftHand-Ac

# Train Label

In [9]:
dfLabel=WFLabelDataFrame(DictionaryDataPath+'07_LabelData.npz',256,32)
TrainLabelDF = pd.concat([dfLabel,dfLabel,dfLabel])

DataAddingLabel is registed now
Build Complete
Source shape is(826837, 1)
window frames num =25831
offset=21
(25831, 256, 1)
DataAddingLabel is registed now
Build Complete
Source shape is(826837, 1)
window frames num =25831
offset=21
(25831, 256, 1)
DataAddingLabel is registed now
Build Complete
Source shape is(826837, 1)
window frames num =25831
offset=21
(25831, 256, 1)
DataAddingLabel is registed now
Build Complete
Source shape is(826837, 1)
window frames num =25831
offset=21
(25831, 256, 1)
DataAddingLabel is registed now
Build Complete
Source shape is(826837, 1)
window frames num =25831
offset=21
(25831, 256, 1)
DataAddingLabel is registed now
Build Complete
Source shape is(826837, 1)
window frames num =25831
offset=21
(25831, 256, 1)
DataAddingLabel is registed now
Build Complete
Source shape is(826837, 1)
window frames num =25831
offset=21
(25831, 256, 1)
DataAddingLabel is registed now
Build Complete
Source shape is(826837, 1)
window frames num =25831
offset=21
(25831, 256, 1)


# Test Label

In [10]:
TestDictionaryDataPath='/media/takeyama/Transfer/02_ActivityResearch/Participants07/03_Third/dictionary/01_First/'

TestLabelDF=WFLabelDataFrame(TestDictionaryDataPath+'07_LabelData.npz',256,32)

DataAddingLabel is registed now
Build Complete
Source shape is(773550, 1)
window frames num =24166
offset=14
(24166, 256, 1)
DataAddingLabel is registed now
Build Complete
Source shape is(773550, 1)
window frames num =24166
offset=14
(24166, 256, 1)
DataAddingLabel is registed now
Build Complete
Source shape is(773550, 1)
window frames num =24166
offset=14
(24166, 256, 1)
DataAddingLabel is registed now
Build Complete
Source shape is(773550, 1)
window frames num =24166
offset=14
(24166, 256, 1)
DataAddingLabel is registed now
Build Complete
Source shape is(773550, 1)
window frames num =24166
offset=14
(24166, 256, 1)
DataAddingLabel is registed now
Build Complete
Source shape is(773550, 1)
window frames num =24166
offset=14
(24166, 256, 1)
DataAddingLabel is registed now
Build Complete
Source shape is(773550, 1)
window frames num =24166
offset=14
(24166, 256, 1)
DataAddingLabel is registed now
Build Complete
Source shape is(773550, 1)
window frames num =24166
offset=14
(24166, 256, 1)


# Normlization

In [11]:
TrainDataDF.Energy = (TrainDataDF.Energy-TrainDataDF.Energy.mean())/( TrainDataDF.Energy.max()-TrainDataDF.Energy.min())
TrainDataDF.Entropy = (TrainDataDF.Entropy-TrainDataDF.Entropy.mean())/( TrainDataDF.Entropy.max()-TrainDataDF.Entropy.min())
TrainDataDF.Max = (TrainDataDF.Max-TrainDataDF.Max.mean())/( TrainDataDF.Max.max()-TrainDataDF.Max.min())
TrainDataDF.Mean = (TrainDataDF.Mean-TrainDataDF.Mean.mean())/( TrainDataDF.Mean.max()-TrainDataDF.Mean.min())
TrainDataDF.Median = (TrainDataDF.Median-TrainDataDF.Median.mean())/( TrainDataDF.Median.max()-TrainDataDF.Median.min())
TrainDataDF.Min = (TrainDataDF.Min-TrainDataDF.Min.mean())/( TrainDataDF.Min.max()-TrainDataDF.Min.min())
TrainDataDF.RMS = (TrainDataDF.RMS-TrainDataDF.RMS.mean())/( TrainDataDF.RMS.max()-TrainDataDF.RMS.min())
TrainDataDF.Range = (TrainDataDF.Range-TrainDataDF.Range.mean())/( TrainDataDF.Range.max()-TrainDataDF.Range.min())
TrainDataDF.StdDev = (TrainDataDF.StdDev-TrainDataDF.StdDev.mean())/( TrainDataDF.StdDev.max()-TrainDataDF.StdDev.min())
TrainDataDF.Var = (TrainDataDF.Var-TrainDataDF.Var.mean())/( TrainDataDF.Var.max()-TrainDataDF.Var.min())

In [12]:
TrainDataDF.head()

,Energy,Entropy,Max,Mean,Median,Min,RMS,Range,StdDev,Var
0,0.025494,-0.094519,0.174103,0.370917,0.426813,0.158555,0.094335,-0.002882,0.025495,-0.000408
1,-0.001029,-0.058223,0.174103,0.385017,0.431473,0.166406,0.112549,-0.008730,-0.001029,-0.005205
2,-0.037345,0.006400,0.174103,0.396175,0.431473,0.181193,0.126888,-0.019745,-0.037345,-0.009504
3,-0.055732,-0.152085,0.174103,0.400602,0.431473,0.203098,0.132982,-0.036063,-0.055733,-0.010680
4,-0.056517,-0.167051,0.174103,0.400768,0.431473,0.206864,0.133216,-0.038868,-0.056519,-0.010715


In [13]:
TestDataDF.Energy = (TestDataDF.Energy-TestDataDF.Energy.mean())/( TestDataDF.Energy.max()-TestDataDF.Energy.min())
TestDataDF.Entropy = (TestDataDF.Entropy-TestDataDF.Entropy.mean())/( TestDataDF.Entropy.max()-TestDataDF.Entropy.min())
TestDataDF.Max = (TestDataDF.Max-TestDataDF.Max.mean())/( TestDataDF.Max.max()-TestDataDF.Max.min())
TestDataDF.Mean = (TestDataDF.Mean-TestDataDF.Mean.mean())/( TestDataDF.Mean.max()-TestDataDF.Mean.min())
TestDataDF.Median = (TestDataDF.Median-TestDataDF.Median.mean())/( TestDataDF.Median.max()-TestDataDF.Median.min())
TestDataDF.Min = (TestDataDF.Min-TestDataDF.Min.mean())/( TestDataDF.Min.max()-TestDataDF.Min.min())
TestDataDF.RMS = (TestDataDF.RMS-TestDataDF.RMS.mean())/( TestDataDF.RMS.max()-TestDataDF.RMS.min())
TestDataDF.Range = (TestDataDF.Range-TestDataDF.Range.mean())/( TestDataDF.Range.max()-TestDataDF.Range.min())
TestDataDF.StdDev = (TestDataDF.StdDev-TestDataDF.StdDev.mean())/( TestDataDF.StdDev.max()-TestDataDF.StdDev.min())
TestDataDF.Var = (TestDataDF.Var-TestDataDF.Var.mean())/( TestDataDF.Var.max()-TestDataDF.Var.min())

In [14]:
TestDataDF.head()

,Energy,Entropy,Max,Mean,Median,Min,RMS,Range,StdDev,Var
0,-0.046322,0.474690,-0.103517,-0.184611,-0.175968,-0.057138,0.197972,-0.042824,-0.046323,-0.004902
1,-0.046276,0.480378,-0.103517,-0.184562,-0.175968,-0.057138,0.197895,-0.042824,-0.046276,-0.004901
2,-0.046143,0.434416,-0.103281,-0.184430,-0.175822,-0.057138,0.197686,-0.042620,-0.046143,-0.004901
3,-0.046110,0.440171,-0.103281,-0.184700,-0.176173,-0.057138,0.198113,-0.042620,-0.046109,-0.004901
4,-0.045824,0.325449,-0.103281,-0.185008,-0.176465,-0.057888,0.198601,-0.042007,-0.045821,-0.004899


In [15]:
TrainData = TrainDataDF.as_matrix()
TestData = TestDataDF.as_matrix()

In [16]:
# SHAPE
print "TrainData.shape="+str(TrainData.shape)
print "TestData.shape="+str(TestData.shape)

TrainData.shape=(77493, 10)
TestData.shape=(72498, 10)


# Random Forest

In [17]:
from sklearn.ensemble import RandomForestClassifier

In [18]:
feat_labels = TrainDataDF.columns[1:]

In [ ]:
forest = RandomForestClassifier(n_estimators=10000, random_state=0)

In [ ]:
forest.fit(TrainDataDF, TrainLabelDF.ActiveNumber.values)